In [1]:
import os


In [2]:
%pwd

'e:\\End-to-End-Project\\Data-Science-first\\datascienceproject\\research'

In [3]:
os.chdir('../')
%pwd

'e:\\End-to-End-Project\\Data-Science-first\\datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path:Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [10]:
from src.datascience.constant import *
from src.datascience.utils.common import create_directories, read_yaml

In [ ]:
class ConfigurationManager:
    def __init__(self, con = CONFIG_FILE_PATH,
                 par= PARAMS_FILE_PATH,
                 sch = SCHEMA_FILE_PATH):
        self.config = read_yaml(con)
        self.parmas  = read_yaml(par)
        self.schema = read_yaml(sch)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.parmas.ElasticNet
        schema  = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])

        modelTrainerconfig = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha= params.alpha,
            l1_ratio = params.l1_ratio,
            target_column= schema.name
        )

        return modelTrainerconfig
        

In [12]:
import os
import pandas as pd
from sklearn.linear_model import ElasticNet
from src.datascience import logger
import joblib

In [21]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_x = pd.read_csv(self.config.train_data_path)
        train_y = train_x.pop(self.config.target_column)

        test_x = pd.read_csv(self.config.test_data_path)
        test_y = test_x.pop(self.config.target_column)

        En = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42) 
        En.fit(train_x, train_y)
        logger.info("Model Trained!")

        joblib.dump(En, os.path.join(self.config.root_dir, self.config.model_name))
        logger.info('Model Dumped!')

In [22]:
try:
    config = ConfigurationManager()
    modelconfig = config.get_model_trainer_config()
    modeltrain = ModelTrainer(modelconfig)
    modeltrain.train()
    
except Exception as e:
    raise e

[ 2025-02-25 15:03:14,999 : INFO : common : yaml file: config\config.yaml loaded successfully ]
[ 2025-02-25 15:03:15,003 : INFO : common : yaml file: params.yaml loaded successfully ]
[ 2025-02-25 15:03:15,008 : INFO : common : yaml file: schema.yaml loaded successfully ]
[ 2025-02-25 15:03:15,010 : INFO : common : created directory at: artifacts ]
[ 2025-02-25 15:03:15,013 : INFO : common : created directory at: artifacts/model_trainer ]
[ 2025-02-25 15:03:15,086 : INFO : 3990415397 : Model Trained! ]
[ 2025-02-25 15:03:15,091 : INFO : 3990415397 : Model Dumped! ]
